In [ ]:
import numpy as np
import pandas as pd
import cv2
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.ndimage import gaussian_filter1d
from sklearn.feature_selection import mutual_info_regression
import warnings
warnings.filterwarnings('ignore')

class SpatialTemporalCrossCorrelation:
    def __init__(self, video_path, output_dir=None, spatial_grid=(4, 4), temporal_segment=1.0):
        """
        Combined Spatial-Temporal Cross-Correlation Analysis
        
        Args:
            video_path: Path to input video
            output_dir: Output directory for results
            spatial_grid: Grid division (rows, cols) for spatial analysis
            temporal_segment: Duration of each temporal segment in seconds
        """
        self.video_path = video_path
        self.video_name = os.path.splitext(os.path.basename(video_path))[0]
        self.output_dir = output_dir or os.path.dirname(video_path)
        self.spatial_grid = spatial_grid
        self.temporal_segment = temporal_segment
        self.num_regions = spatial_grid[0] * spatial_grid[1]
        
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Analysis storage
        self.video_info = {}
        self.movement_matrix = None  # Regions × Time segments
        self.bitrate_matrix = None   # Regions × Time segments
        self.spatial_temporal_cross_corr = None
        self.combined_correlation_matrices = {}
        self.temporal_delay_analysis = {}  # New: Store temporal delay analysis
        
    def extract_video_information(self):
        """Extract basic video metadata"""
        print("Extracting video information...")
        
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {self.video_path}")
        
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps if fps > 0 else 0
        
        self.video_info = {
            'width': width,
            'height': height, 
            'fps': fps,
            'total_frames': total_frames,
            'duration': duration,
            'region_width': width // self.spatial_grid[1],
            'region_height': height // self.spatial_grid[0],
            'frames_per_segment': int(fps * self.temporal_segment),
            'total_segments': int(duration / self.temporal_segment)
        }
        
        cap.release()
        print(f"Video: {width}x{height}, {fps:.1f} fps, {duration:.1f}s, {total_frames} frames")
        print(f"Grid: {self.spatial_grid[0]}x{self.spatial_grid[1]} = {self.num_regions} regions")
        print(f"Temporal: {self.temporal_segment}s segments → {self.video_info['total_segments']} segments")
        
        return self.video_info
    
    def compute_movement_intensity(self):
        """
        Compute movement intensity for each spatial region over time segments
        Returns: movement_matrix (regions × time_segments)
        """
        print("Computing spatial-temporal movement intensity...")
        
        cap = cv2.VideoCapture(self.video_path)
        fps = self.video_info['fps']
        frames_per_segment = self.video_info['frames_per_segment']
        total_segments = self.video_info['total_segments']
        
        # Initialize movement matrix
        self.movement_matrix = np.zeros((self.num_regions, total_segments))
        
        # Motion detection setup
        backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=25)
        prev_frame = None
        
        for segment_idx in range(total_segments):
            segment_movement = np.zeros(self.num_regions)
            frames_processed = 0
            
            for frame_idx in range(frames_per_segment):
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Convert to grayscale
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                # Background subtraction
                fg_mask = backSub.apply(frame)
                fg_mask = cv2.medianBlur(fg_mask, 5)
                
                # Optical flow for precise motion
                if prev_frame is not None:
                    flow = cv2.calcOpticalFlowFarneback(
                        prev_frame, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0
                    )
                    flow_magnitude = np.sqrt(flow[..., 0]**2 + flow[..., 1]**2)
                else:
                    flow_magnitude = np.zeros_like(gray, dtype=np.float32)
                
                # Calculate movement for each region
                for region_idx in range(self.num_regions):
                    row = region_idx // self.spatial_grid[1]
                    col = region_idx % self.spatial_grid[1]
                    
                    y_start = row * self.video_info['region_height']
                    y_end = min((row + 1) * self.video_info['region_height'], self.video_info['height'])
                    x_start = col * self.video_info['region_width'] 
                    x_end = min((col + 1) * self.video_info['region_width'], self.video_info['width'])
                    
                    # Extract region from mask and flow
                    region_mask = fg_mask[y_start:y_end, x_start:x_end]
                    region_flow = flow_magnitude[y_start:y_end, x_start:x_end]
                    
                    # Combined movement intensity
                    mask_intensity = np.sum(region_mask > 128)
                    flow_intensity = np.sum(region_flow)
                    region_area = (y_end - y_start) * (x_end - x_start)
                    
                    if region_area > 0:
                        movement = (0.7 * mask_intensity + 0.3 * flow_intensity) / region_area
                        segment_movement[region_idx] += movement
                
                prev_frame = gray.copy()
                frames_processed += 1
            
            if frames_processed > 0:
                self.movement_matrix[:, segment_idx] = segment_movement / frames_processed
            
            if segment_idx % 10 == 0:
                progress = (segment_idx / total_segments) * 100
                print(f"Progress: {progress:.1f}% - Segment {segment_idx}/{total_segments}")
        
        cap.release()
        
        # Smooth temporal patterns
        self.movement_matrix = gaussian_filter1d(self.movement_matrix, sigma=1, axis=1)
        
        # Normalize each region
        for i in range(self.num_regions):
            if np.max(self.movement_matrix[i]) > np.min(self.movement_matrix[i]):
                self.movement_matrix[i] = (self.movement_matrix[i] - np.min(self.movement_matrix[i])) / \
                                        (np.max(self.movement_matrix[i]) - np.min(self.movement_matrix[i]))
        
        print(f"Movement matrix shape: {self.movement_matrix.shape}")
        return self.movement_matrix

    def compute_temporal_delay_correlation(self, max_time_lag=10):
        """
        Compute correlation between regions with temporal delays
        Analyzes correlation at different time lags from -max_time_lag to +max_time_lag seconds
        
        Args:
            max_time_lag: Maximum time lag in seconds to analyze (positive and negative)
        """
        print(f"Computing temporal delay correlation analysis (time lags: -{max_time_lag} to +{max_time_lag} seconds)...")
        
        if self.movement_matrix is None:
            self.compute_movement_intensity()
        
        num_regions = self.movement_matrix.shape[0]
        num_segments = self.movement_matrix.shape[1]
        
        # Convert time lag to segment lag
        max_lag_segments = int(max_time_lag / self.temporal_segment)
        total_lags = 2 * max_lag_segments + 1
        
        print(f"Time lag: ±{max_time_lag}s → Segment lag: ±{max_lag_segments} segments")
        
        # Initialize storage for temporal delay analysis
        self.temporal_delay_analysis = {
            'max_correlation_matrix': np.zeros((num_regions, num_regions)),
            'optimal_time_lag_matrix': np.zeros((num_regions, num_regions)),  # in seconds
            'optimal_segment_lag_matrix': np.zeros((num_regions, num_regions)),  # in segments
            'lag_correlations': np.zeros((num_regions, num_regions, total_lags)),
            'significant_correlations': np.zeros((num_regions, num_regions), dtype=bool),
            'high_significant_correlations': np.zeros((num_regions, num_regions), dtype=bool),  # New: for corr > 0.5
            'time_lags_seconds': np.linspace(-max_time_lag, max_time_lag, total_lags),
            'segment_lags': np.arange(-max_lag_segments, max_lag_segments + 1),
            'max_time_lag': max_time_lag,
            'max_lag_segments': max_lag_segments
        }
        
        # Define significance thresholds
        significance_threshold = 0.3
        high_significance_threshold = 0.5  # New: for high correlation
        
        for i in range(num_regions):
            for j in range(num_regions):
                if i == j:
                    continue  # Skip auto-correlation
                
                signal_i = self.movement_matrix[i]
                signal_j = self.movement_matrix[j]
                
                # Remove DC component and normalize
                signal_i_norm = (signal_i - np.mean(signal_i)) / (np.std(signal_i) + 1e-8)
                signal_j_norm = (signal_j - np.mean(signal_j)) / (np.std(signal_j) + 1e-8)
                
                # Compute cross-correlation for different time lags
                lag_correlations = []
                
                for lag_idx, segment_lag in enumerate(self.temporal_delay_analysis['segment_lags']):
                    if segment_lag < 0:
                        # Region i leads region j (negative lag: i before j)
                        corr_signal_i = signal_i_norm[:segment_lag] if segment_lag != 0 else signal_i_norm
                        corr_signal_j = signal_j_norm[-segment_lag:]
                    elif segment_lag > 0:
                        # Region j leads region i (positive lag: j before i)
                        corr_signal_i = signal_i_norm[segment_lag:]
                        corr_signal_j = signal_j_norm[:-segment_lag] if segment_lag != 0 else signal_j_norm
                    else:
                        # No lag
                        corr_signal_i = signal_i_norm
                        corr_signal_j = signal_j_norm
                    
                    # Ensure same length
                    min_len = min(len(corr_signal_i), len(corr_signal_j))
                    if min_len > 10:  # Need sufficient data points
                        corr_coef = np.corrcoef(corr_signal_i[:min_len], corr_signal_j[:min_len])[0, 1]
                        # Handle NaN values
                        if np.isnan(corr_coef):
                            corr_coef = 0
                        lag_correlations.append(corr_coef)
                    else:
                        lag_correlations.append(0)
                
                # Store lag correlations
                self.temporal_delay_analysis['lag_correlations'][i, j] = lag_correlations
                
                # Find maximum correlation and optimal lag
                max_corr_idx = np.argmax(np.abs(lag_correlations))
                max_correlation = lag_correlations[max_corr_idx]
                optimal_segment_lag = self.temporal_delay_analysis['segment_lags'][max_corr_idx]
                optimal_time_lag = self.temporal_delay_analysis['time_lags_seconds'][max_corr_idx]
                
                self.temporal_delay_analysis['max_correlation_matrix'][i, j] = max_correlation
                self.temporal_delay_analysis['optimal_segment_lag_matrix'][i, j] = optimal_segment_lag
                self.temporal_delay_analysis['optimal_time_lag_matrix'][i, j] = optimal_time_lag
                
                # Check if correlation is significant
                if abs(max_correlation) >= significance_threshold:
                    self.temporal_delay_analysis['significant_correlations'][i, j] = True
                
                # Check if correlation is highly significant (new)
                if abs(max_correlation) >= high_significance_threshold:
                    self.temporal_delay_analysis['high_significant_correlations'][i, j] = True
            
            # Progress update
            if (i + 1) % 5 == 0:
                print(f"Progress: {i+1}/{num_regions} regions completed")
        
        print("Temporal delay correlation analysis completed")
        return self.temporal_delay_analysis

    def analyze_exact_time_windows(self, correlation_threshold=0.5, window_size_seconds=10):
        """
        Analyze EXACT time windows (0-10s, 10-20s, etc.) where high correlations exist
        between specific region pairs
        
        Args:
            correlation_threshold: Minimum correlation to consider as high
            window_size_seconds: Size of time windows in seconds
        """
        print(f"\nAnalyzing EXACT time windows ({window_size_seconds}-second intervals) with high correlations (> {correlation_threshold})...")
        
        if self.movement_matrix is None:
            self.compute_movement_intensity()
        
        if not self.temporal_delay_analysis:
            self.compute_temporal_delay_correlation(max_time_lag=10)
        
        num_segments = self.movement_matrix.shape[1]
        segments_per_window = int(window_size_seconds / self.temporal_segment)
        num_windows = num_segments // segments_per_window
        
        print(f"Time windows: {num_windows} windows of {window_size_seconds} seconds each")
        print(f"Total analysis duration: {num_windows * window_size_seconds} seconds")
        
        # Store time window analysis results
        time_window_analysis = {
            'window_size_seconds': window_size_seconds,
            'correlation_threshold': correlation_threshold,
            'windows': [],
            'high_correlation_events': []
        }
        
        # Analyze each time window
        for window_idx in range(num_windows):
            start_segment = window_idx * segments_per_window
            end_segment = min((window_idx + 1) * segments_per_window, num_segments)
            start_time = window_idx * window_size_seconds
            end_time = (window_idx + 1) * window_size_seconds
            
            window_data = {
                'window_id': window_idx,
                'start_time': start_time,
                'end_time': end_time,
                'start_segment': start_segment,
                'end_segment': end_segment,
                'high_correlation_pairs': []
            }
            
            # Find high correlation pairs in this window
            for i in range(self.num_regions):
                for j in range(self.num_regions):
                    if i >= j:  # Avoid duplicates and self-correlation
                        continue
                    
                    # Extract signals for this window
                    signal_i_window = self.movement_matrix[i, start_segment:end_segment]
                    signal_j_window = self.movement_matrix[j, start_segment:end_segment]
                    
                    # Compute correlation for this specific window
                    if len(signal_i_window) > 5 and len(signal_j_window) > 5:  # Need sufficient data
                        window_correlation = np.corrcoef(signal_i_window, signal_j_window)[0, 1]
                        
                        if not np.isnan(window_correlation) and abs(window_correlation) >= correlation_threshold:
                            # Found high correlation in this window
                            pair_data = {
                                'region_i': i,
                                'region_j': j,
                                'correlation': float(window_correlation),
                                'direction': 'positive' if window_correlation >= 0 else 'negative'
                            }
                            window_data['high_correlation_pairs'].append(pair_data)
                            
                            # Also store as event
                            time_window_analysis['high_correlation_events'].append({
                                'time_window': f"{start_time}-{end_time}s",
                                'start_time_seconds': start_time,
                                'end_time_seconds': end_time,
                                'region_i': i,
                                'region_j': j,
                                'correlation': float(window_correlation),
                                'absolute_correlation': float(abs(window_correlation)),
                                'direction': 'positive' if window_correlation >= 0 else 'negative'
                            })
            
            time_window_analysis['windows'].append(window_data)
        
        # Sort high correlation events by correlation strength
        time_window_analysis['high_correlation_events'].sort(
            key=lambda x: x['absolute_correlation'], reverse=True
        )
        
        self.time_window_analysis = time_window_analysis
        print(f"Found {len(time_window_analysis['high_correlation_events'])} high correlation events across {num_windows} time windows")
        
        return time_window_analysis

    def create_exact_time_heatmaps(self, correlation_threshold=0.5, window_size_seconds=10):
        """
        Create heatmaps showing EXACT time durations when high correlations occur
        between specific region pairs
        """
        print(f"Creating EXACT time duration heatmaps ({window_size_seconds}-second windows)...")
        
        # Perform time window analysis
        time_analysis = self.analyze_exact_time_windows(correlation_threshold, window_size_seconds)
        
        # Create output directory for exact time heatmaps
        exact_time_dir = os.path.join(self.output_dir, "exact_time_correlation_heatmaps")
        os.makedirs(exact_time_dir, exist_ok=True)
        
        # HEATMAP 1: Time Windows vs Region Pairs
        self._create_time_window_heatmap(exact_time_dir, time_analysis, window_size_seconds)
        
        # HEATMAP 2: Temporal Distribution of High Correlations
        self._create_temporal_distribution_heatmap(exact_time_dir, time_analysis, window_size_seconds)
        
        # HEATMAP 3: Region Pair Activity Timeline
        self._create_region_timeline_heatmap(exact_time_dir, time_analysis, window_size_seconds)
        
        # Generate detailed time window report
        self._generate_time_window_report(exact_time_dir, time_analysis)
        
        print(f"All exact time heatmaps saved in: {exact_time_dir}")
        return exact_time_dir

    def _create_time_window_heatmap(self, output_dir, time_analysis, window_size_seconds):
        """
        Create heatmap showing which region pairs have high correlations in which time windows
        """
        print("Creating time window vs region pairs heatmap...")
        
        # Create matrix: time_windows × region_pairs
        num_windows = len(time_analysis['windows'])
        num_pairs = self.num_regions * (self.num_regions - 1) // 2  # Unique pairs
        
        # Initialize heatmap matrix
        heatmap_matrix = np.zeros((num_windows, num_pairs))
        pair_labels = []
        
        # Create mapping from (i,j) to pair index
        pair_index_map = {}
        pair_idx = 0
        for i in range(self.num_regions):
            for j in range(i+1, self.num_regions):
                pair_index_map[(i, j)] = pair_idx
                pair_labels.append(f"R{i}-R{j}")
                pair_idx += 1
        
        # Fill heatmap matrix
        for window_idx, window_data in enumerate(time_analysis['windows']):
            for pair_data in window_data['high_correlation_pairs']:
                i = pair_data['region_i']
                j = pair_data['region_j']
                pair_key = (min(i, j), max(i, j))  # Ensure consistent ordering
                if pair_key in pair_index_map:
                    heatmap_matrix[window_idx, pair_index_map[pair_key]] = pair_data['correlation']
        
        # Create the heatmap
        plt.figure(figsize=(20, 12))
        
        # Use only pairs that have at least one high correlation
        active_pairs_mask = np.any(heatmap_matrix != 0, axis=0)
        active_heatmap = heatmap_matrix[:, active_pairs_mask]
        active_labels = [pair_labels[i] for i in range(len(pair_labels)) if active_pairs_mask[i]]
        
        if active_heatmap.size == 0:
            print("No high correlation pairs found for heatmap")
            return
        
        im = plt.imshow(active_heatmap.T, aspect='auto', cmap='RdYlBu_r', 
                       vmin=-1, vmax=1, interpolation='nearest')
        
        # Set labels
        time_labels = [f"{w['start_time']}-{w['end_time']}s" for w in time_analysis['windows']]
        plt.xticks(np.arange(len(time_labels)), time_labels, rotation=45, ha='right')
        plt.yticks(np.arange(len(active_labels)), active_labels)
        
        plt.xlabel(f'Time Windows ({window_size_seconds} seconds each)', fontweight='bold', fontsize=12)
        plt.ylabel('Region Pairs', fontweight='bold', fontsize=12)
        plt.title(f'EXACT TIME CORRELATION HEATMAP\n{self.video_name}\n'
                 f'High Correlations (> {time_analysis["correlation_threshold"]}) in {window_size_seconds}-Second Windows',
                 fontweight='bold', fontsize=14, pad=20)
        
        # Add colorbar
        cbar = plt.colorbar(im, label='Correlation Coefficient')
        cbar.set_label('Correlation Coefficient', fontweight='bold')
        
        # Add grid
        plt.grid(True, alpha=0.3, color='black')
        
        plt.tight_layout()
        heatmap_path = os.path.join(output_dir, f'{self.video_name}_exact_time_correlation_heatmap.png')
        plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ Exact time correlation heatmap saved: {heatmap_path}")

    def _create_temporal_distribution_heatmap(self, output_dir, time_analysis, window_size_seconds):
        """
        Create heatmap showing temporal distribution of high correlations
        """
        print("Creating temporal distribution heatmap...")
        
        # Count high correlations per time window
        high_corr_counts = [len(window['high_correlation_pairs']) for window in time_analysis['windows']]
        time_windows = [f"{w['start_time']}-{w['end_time']}s" for w in time_analysis['windows']]
        
        # Create the plot
        plt.figure(fsize=(16, 8))
        
        bars = plt.bar(range(len(high_corr_counts)), high_corr_counts, 
                      color='red', alpha=0.7, edgecolor='darkred')
        
        plt.xlabel(f'Time Windows ({window_size_seconds} seconds each)', fontweight='bold', fontsize=12)
        plt.ylabel('Number of High Correlation Pairs', fontweight='bold', fontsize=12)
        plt.title(f'TEMPORAL DISTRIBUTION OF HIGH CORRELATIONS\n{self.video_name}\n'
                 f'Number of Region Pairs with Correlation > {time_analysis["correlation_threshold"]} in Each Time Window',
                 fontweight='bold', fontsize=14, pad=20)
        
        plt.xticks(range(len(time_windows)), time_windows, rotation=45, ha='right')
        
        # Add value labels on bars
        for bar, count in zip(bars, high_corr_counts):
            if count > 0:
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                        str(count), ha='center', va='bottom', fontweight='bold')
        
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        
        dist_path = os.path.join(output_dir, f'{self.video_name}_temporal_distribution.png')
        plt.savefig(dist_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ Temporal distribution heatmap saved: {dist_path}")

    def _create_region_timeline_heatmap(self, output_dir, time_analysis, window_size_seconds):
        """
        Create heatmap showing when each region is active in high correlations
        """
        print("Creating region activity timeline heatmap...")
        
        # Create matrix: regions × time_windows
        activity_matrix = np.zeros((self.num_regions, len(time_analysis['windows'])))
        
        for window_idx, window_data in enumerate(time_analysis['windows']):
            for pair_data in window_data['high_correlation_pairs']:
                i = pair_data['region_i']
                j = pair_data['region_j']
                activity_matrix[i, window_idx] += 1
                activity_matrix[j, window_idx] += 1
        
        # Create the heatmap
        plt.figure(figsize=(18, 10))
        
        im = plt.imshow(activity_matrix, aspect='auto', cmap='YlOrRd', 
                       interpolation='nearest', vmin=0, vmax=np.max(activity_matrix))
        
        # Set labels
        time_labels = [f"{w['start_time']}-{w['end_time']}s" for w in time_analysis['windows']]
        region_labels = [f"R{i}" for i in range(self.num_regions)]
        
        plt.xticks(np.arange(len(time_labels)), time_labels, rotation=45, ha='right')
        plt.yticks(np.arange(len(region_labels)), region_labels)
        
        plt.xlabel(f'Time Windows ({window_size_seconds} seconds each)', fontweight='bold', fontsize=12)
        plt.ylabel('Regions', fontweight='bold', fontsize=12)
        plt.title(f'REGION ACTIVITY TIMELINE\n{self.video_name}\n'
                 f'Number of High Correlation Involvements per Region in Each Time Window',
                 fontweight='bold', fontsize=14, pad=20)
        
        # Add colorbar
        cbar = plt.colorbar(im, label='Number of High Correlation Involvements')
        cbar.set_label('Number of High Correlation Involvements', fontweight='bold')
        
        # Add values in cells
        for i in range(activity_matrix.shape[0]):
            for j in range(activity_matrix.shape[1]):
                if activity_matrix[i, j] > 0:
                    plt.text(j, i, f'{int(activity_matrix[i, j])}', 
                            ha="center", va="center", color="white" if activity_matrix[i, j] > np.max(activity_matrix)/2 else "black",
                            fontweight='bold')
        
        plt.grid(True, alpha=0.3, color='black')
        plt.tight_layout()
        
        timeline_path = os.path.join(output_dir, f'{self.video_name}_region_activity_timeline.png')
        plt.savefig(timeline_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ Region activity timeline saved: {timeline_path}")

    def _generate_time_window_report(self, output_dir, time_analysis):
        """
        Generate detailed report showing exact time windows with high correlations
        """
        print("Generating detailed time window report...")
        
        # Create summary statistics
        total_events = len(time_analysis['high_correlation_events'])
        active_windows = sum(1 for window in time_analysis['windows'] if window['high_correlation_pairs'])
        
        # Find most active time windows
        window_activity = []
        for window in time_analysis['windows']:
            if window['high_correlation_pairs']:
                max_corr = max([abs(pair['correlation']) for pair in window['high_correlation_pairs']])
                window_activity.append({
                    'time_window': f"{window['start_time']}-{window['end_time']}s",
                    'start_time': window['start_time'],
                    'pair_count': len(window['high_correlation_pairs']),
                    'max_correlation': max_corr
                })
        
        # Sort by activity
        window_activity.sort(key=lambda x: x['pair_count'], reverse=True)
        
        # Find strongest correlations
        strong_events = sorted(time_analysis['high_correlation_events'], 
                              key=lambda x: x['absolute_correlation'], reverse=True)[:20]
        
        # Compile report
        time_report = {
            'summary': {
                'total_high_correlation_events': total_events,
                'total_time_windows': len(time_analysis['windows']),
                'active_time_windows': active_windows,
                'correlation_threshold': time_analysis['correlation_threshold'],
                'window_size_seconds': time_analysis['window_size_seconds'],
                'percentage_active_windows': (active_windows / len(time_analysis['windows'])) * 100
            },
            'most_active_windows': window_activity[:10],
            'strongest_correlations': [
                {
                    'time_window': event['time_window'],
                    'region_pair': f"R{event['region_i']}-R{event['region_j']}",
                    'correlation': event['correlation'],
                    'absolute_correlation': event['absolute_correlation'],
                    'direction': event['direction']
                }
                for event in strong_events
            ],
            'detailed_windows': []
        }
        
        # Add detailed window information (only active windows)
        for window in time_analysis['windows']:
            if window['high_correlation_pairs']:
                window_detail = {
                    'time_window': f"{window['start_time']}-{window['end_time']}s",
                    'start_time_seconds': window['start_time'],
                    'end_time_seconds': window['end_time'],
                    'high_correlation_pairs': [
                        {
                            'region_pair': f"R{pair['region_i']}-R{pair['region_j']}",
                            'correlation': pair['correlation'],
                            'direction': pair['direction']
                        }
                        for pair in window['high_correlation_pairs']
                    ]
                }
                time_report['detailed_windows'].append(window_detail)
        
        # Save JSON report
        report_path = os.path.join(output_dir, f'{self.video_name}_exact_time_report.json')
        with open(report_path, 'w') as f:
            json.dump(time_report, f, indent=2)
        
        # Save CSV report for easy analysis
        csv_data = []
        for event in time_analysis['high_correlation_events']:
            csv_data.append({
                'Time_Window': event['time_window'],
                'Start_Time_Seconds': event['start_time_seconds'],
                'End_Time_Seconds': event['end_time_seconds'],
                'Region_I': event['region_i'],
                'Region_J': event['region_j'],
                'Region_Pair': f"R{event['region_i']}-R{event['region_j']}",
                'Correlation': event['correlation'],
                'Absolute_Correlation': event['absolute_correlation'],
                'Direction': event['direction']
            })
        
        if csv_data:
            df = pd.DataFrame(csv_data)
            csv_path = os.path.join(output_dir, f'{self.video_name}_exact_time_correlations.csv')
            df.to_csv(csv_path, index=False)
        
        # Print summary
        self._print_time_window_summary(time_report)
        
        print(f"✓ Detailed time window report saved: {report_path}")
        if csv_data:
            print(f"✓ CSV data saved: {csv_path}")
        
        return time_report

    def _print_time_window_summary(self, time_report):
        """Print summary of time window analysis"""
        print("\n" + "="*80)
        print("EXACT TIME WINDOW ANALYSIS SUMMARY")
        print("="*80)
        
        summary = time_report['summary']
        print(f"Total High Correlation Events: {summary['total_high_correlation_events']}")
        print(f"Active Time Windows: {summary['active_time_windows']}/{summary['total_time_windows']} ({summary['percentage_active_windows']:.1f}%)")
        print(f"Correlation Threshold: > {summary['correlation_threshold']}")
        print(f"Window Size: {summary['window_size_seconds']} seconds")
        
        if time_report['most_active_windows']:
            print(f"\nMost Active Time Windows:")
            for i, window in enumerate(time_report['most_active_windows'][:5], 1):
                print(f"  {i}. {window['time_window']}: {window['pair_count']} high correlation pairs (max: {window['max_correlation']:.3f})")
        
        if time_report['strongest_correlations']:
            print(f"\nStrongest Correlations:")
            for i, corr in enumerate(time_report['strongest_correlations'][:5], 1):
                print(f"  {i}. {corr['time_window']} - {corr['region_pair']}: {corr['correlation']:.3f}")
        
        print("="*80)

    def generate_comprehensive_temporal_report(self):
        """
        Generate comprehensive report including exact time window analysis
        """
        print("\n" + "="*80)
        print("GENERATING COMPREHENSIVE TEMPORAL ANALYSIS REPORT")
        print("="*80)
        
        # Ensure all analyses are performed
        self.extract_video_information()
        self.compute_movement_intensity()
        
        # Perform temporal delay analysis
        self.compute_temporal_delay_correlation(max_time_lag=10)
        
        # Create exact time window analysis
        exact_time_dir = self.create_exact_time_heatmaps(correlation_threshold=0.5, window_size_seconds=10)
        
        # Calculate comprehensive statistics
        if self.temporal_delay_analysis:
            significant_pairs = np.sum(self.temporal_delay_analysis['significant_correlations'])
            high_significant_pairs = np.sum(self.temporal_delay_analysis['high_significant_correlations'])
            
            # Get time window analysis results
            time_report = self.time_window_analysis
            
            # Compile comprehensive report
            comprehensive_report = {
                'video_metadata': self.video_info,
                'analysis_parameters': {
                    'spatial_grid': f"{self.spatial_grid[0]}x{self.spatial_grid[1]}",
                    'temporal_segment_seconds': self.temporal_segment,
                    'total_regions': self.num_regions,
                    'total_segments': self.movement_matrix.shape[1],
                    'max_time_lag_seconds': 10,
                    'significance_threshold': 0.3,
                    'high_significance_threshold': 0.5,
                    'time_window_size_seconds': 10
                },
                'global_correlation_results': {
                    'significant_correlation_pairs': int(significant_pairs),
                    'high_significant_correlation_pairs': int(high_significant_pairs),
                    'total_possible_pairs': self.num_regions * (self.num_regions - 1),
                    'average_correlation_strength': float(np.mean(np.abs(self.temporal_delay_analysis['max_correlation_matrix']))),
                    'average_absolute_time_lag_seconds': float(np.mean(np.abs(self.temporal_delay_analysis['optimal_time_lag_matrix']))),
                    'percentage_significant_pairs': float(significant_pairs / (self.num_regions * (self.num_regions - 1)) * 100),
                    'percentage_high_significant_pairs': float(high_significant_pairs / (self.num_regions * (self.num_regions - 1)) * 100)
                },
                'time_window_analysis': time_report
            }
            
            # Save comprehensive report
            report_path = os.path.join(self.output_dir, f'{self.video_name}_comprehensive_temporal_report.json')
            with open(report_path, 'w') as f:
                json.dump(comprehensive_report, f, indent=2)
            
            print(f"Comprehensive temporal report saved: {report_path}")
            
            # Print comprehensive summary
            self._print_comprehensive_summary(comprehensive_report)
            
            return comprehensive_report

    def _print_comprehensive_summary(self, report):
        """Print comprehensive summary"""
        print("\n" + "="*80)
        print("COMPREHENSIVE TEMPORAL ANALYSIS SUMMARY")
        print("="*80)
        
        global_results = report['global_correlation_results']
        time_window = report['time_window_analysis']['summary']
        
        print(f"Video Duration: {report['video_metadata']['duration']:.1f}s")
        print(f"Spatial Grid: {report['analysis_parameters']['spatial_grid']}")
        print(f"Time Window Size: {report['analysis_parameters']['time_window_size_seconds']}s")
        
        print(f"\nGLOBAL CORRELATION RESULTS:")
        print(f"  High Correlation Pairs (>0.5): {global_results['high_significant_correlation_pairs']} ({global_results['percentage_high_significant_pairs']:.1f}%)")
        print(f"  Average Correlation: {global_results['average_correlation_strength']:.3f}")
        print(f"  Average Time Lag: {global_results['average_absolute_time_lag_seconds']:.2f}s")
        
        print(f"\nEXACT TIME WINDOW ANALYSIS:")
        print(f"  Total High Correlation Events: {time_window['total_high_correlation_events']}")
        print(f"  Active Windows: {time_window['active_time_windows']}/{time_window['total_time_windows']} ({time_window['percentage_active_windows']:.1f}%)")
        
        # Show top time windows with high correlations
        if report['time_window_analysis']['most_active_windows']:
            print(f"\nTOP ACTIVE TIME WINDOWS:")
            for i, window in enumerate(report['time_window_analysis']['most_active_windows'][:3], 1):
                print(f"  {i}. {window['time_window']}: {window['pair_count']} high correlation pairs")
        
        print("="*80)

# Main execution function with proper video discovery
def run_comprehensive_temporal_analysis(video_path, output_dir=None, spatial_grid=(4, 4), temporal_segment=1.0):
    """
    Run complete comprehensive temporal analysis including exact time windows
    """
    analyzer = SpatialTemporalCrossCorrelation(video_path, output_dir, spatial_grid, temporal_segment)
    
    try:
        print(f"Starting COMPREHENSIVE temporal analysis for: {analyzer.video_name}")
        print(f"Including EXACT TIME WINDOW analysis (0-10s, 10-20s, etc.)")
        
        # Run comprehensive analysis
        report = analyzer.generate_comprehensive_temporal_report()
        
        return analyzer, report
        
    except Exception as e:
        print(f"Comprehensive temporal analysis failed: {e}")
        raise

# Function to find and process videos in your folder
def find_and_process_videos(video_folder, output_base_dir="comprehensive_temporal_results"):
    """
    Find all videos in the specified folder and process them
    """
    print(f"Looking for videos in: {video_folder}")
    
    # Check if folder exists
    if not os.path.exists(video_folder):
        print(f"❌ Error: Video folder '{video_folder}' does not exist!")
        return []
    
    # Supported video formats
    video_extensions = ('.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv', '.webm', '.MP4', '.AVI', '.MKV', '.MOV')
    
    # Find all video files
    video_files = []
    for file in os.listdir(video_folder):
        if file.lower().endswith(video_extensions):
            video_files.append(file)
    
    if not video_files:
        print(f"❌ No video files found in {video_folder}")
        print(f"Supported formats: {video_extensions}")
        return []
    
    print(f"✅ Found {len(video_files)} video files:")
    for video_file in video_files:
        print(f"  - {video_file}")
    
    # Process each video
    results = []
    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        output_dir = os.path.join(output_base_dir, video_name)
        
        print(f"\n" + "="*80)
        print(f"PROCESSING: {video_file}")
        print("="*80)
        
        try:
            analyzer, report = run_comprehensive_temporal_analysis(
                video_path=video_path,
                output_dir=output_dir,
                spatial_grid=(4, 4),
                temporal_segment=1.0
            )
            results.append((video_file, analyzer, report))
            print(f"✅ Successfully processed: {video_file}")
            
        except Exception as e:
            print(f"❌ Failed to process {video_file}: {e}")
            continue
    
    return results

# Example usage with your actual folder path
if __name__ == "__main__":
    # Use your actual video folder path
    video_folder = "/mnt/d/TileClipper_Implementation/data/video"
    output_base_dir = "comprehensive_temporal_results"
    
    print("="*80)
    print("SPATIAL-TEMPORAL CORRELATION ANALYSIS")
    print("="*80)
    print(f"Video folder: {video_folder}")
    print(f"Output directory: {output_base_dir}")
    print("="*80)
    
    # Find and process all videos
    results = find_and_process_videos(video_folder, output_base_dir)
    
    if results:
        print(f"\n🎉 ANALYSIS COMPLETED SUCCESSFULLY!")
        print(f"✅ Processed {len(results)} videos")
        print(f"📁 Results saved in: {output_base_dir}")
        
        # Print summary for each video
        print(f"\n" + "="*80)
        print("SUMMARY OF RESULTS")
        print("="*80)
        
        for video_file, analyzer, report in results:
            print(f"\n📊 {video_file}:")
            if hasattr(analyzer, 'time_window_analysis'):
                time_report = analyzer.time_window_analysis['summary']
                print(f"  - Duration: {analyzer.video_info['duration']:.1f}s")
                print(f"  - High correlation events: {time_report['total_high_correlation_events']}")
                print(f"  - Active time windows: {time_report['active_time_windows']}/{time_report['total_time_windows']}")
                
                # Show specific time windows with high correlations
                active_windows = [w for w in analyzer.time_window_analysis['windows'] if w['high_correlation_pairs']]
                if active_windows:
                    print(f"  - Top time windows with high correlations:")
                    for window in active_windows[:3]:  # Show first 3 active windows
                        print(f"    • {window['start_time']}-{window['end_time']}s: {len(window['high_correlation_pairs'])} pairs")
    else:
        print(f"\n❌ No videos were successfully processed.")
        print(f"Please check:")
        print(f"  1. The folder path: {video_folder}")
        print(f"  2. Video file formats (supported: mp4, avi, mkv, mov, wmv, flv, webm)")
        print(f"  3. File permissions")